# Wordle

### Strategy 1
* Find the most common letters in words with N letters
* Find words without any repeating letters made of the most common letters
* Submit first guess and plug in learning from interactive cli

In [256]:
# Load the autoreload extension
%load_ext autoreload

# Autoreload reloads modules before executing code
# 0: disable
# 1: reload modules imported with %aimport
# 2: reload all modules, except those excluded by %aimport
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [257]:
import os
import nltk

from nltk.corpus import words

SRC_DIR = os.path.realpath(os.path.join(os.path.curdir, '..'))
NLTK_DIR = os.path.join(SRC_DIR, 'src/makewords/nltk_data')
nltk.data.path.append(NLTK_DIR)

In [258]:
# Prepare for analysis

import string
from itertools import chain

import numpy as np
import pandas as pd

N = 5  # target words with this number of letters
zs = ['z{}'.format(n) for n in range(N)]  # for counting appearances, index 0-5
ns = ['n{}'.format(n) for n in range(1,N)]  # for counting n appearances per word

def get_clean_df():
    df = pd.DataFrame(
        0,
        index=np.arange(26),
        columns=list(chain(['total'], list(zs), list(ns)))
    )
    df.insert(
        0,
        'letters',
        list(string.ascii_lowercase)
    )
    df = df.set_index('letters')
    return df

df = get_clean_df()
df.head(4)

,total,z0,z1,z2,z3,z4,n1,n2,n3,n4
letters,,,,,,,,,,
a,0,0,0,0,0,0,0,0,0,0
b,0,0,0,0,0,0,0,0,0,0
c,0,0,0,0,0,0,0,0,0,0
d,0,0,0,0,0,0,0,0,0,0


In [259]:
# Begin analysis
import collections

# TODO Clean and cache this
nl_words = [word.lower() for word in words.words() if len(word)==N]

def get_possible_words(df, letters=None, words=None):
    for word in nl_words:
        if letters is not None:
            for letter in letters:
                if letter not in word:
                    continue
        for i,j in enumerate(word):
            df.loc[j]['total'] += 1          # count the word once
            df.loc[j]['z{}'.format(i)] += 1  # count each register
        letter_count = dict(collections.Counter(word))
        for letter,n in letter_count.items():
            df.loc[letter]['n{}'.format(n)] += 1  # count appearances per word
    return df

df = get_possible_words(df)
df.head(4)

,total,z0,z1,z2,z3,z4,n1,n2,n3,n4
letters,,,,,,,,,,
a,5879,810,1837,1040,1162,1030,4167,805,34,0
b,1397,750,66,316,189,76,1185,100,4,0
c,1878,768,175,292,454,189,1664,104,2,0
d,1703,467,90,364,313,469,1499,96,4,0


In [260]:
def first_guess(df):
    # Take the most common letters
    chars = (
        df['total']
        .sort_values(axis='index', ascending=False)
        # .head(N)
        .keys()
        .to_list()
    )

    # Find words containing the most common letters with no repeats
    occurrences_by_register = (
        df.sort_values(by='total', ascending=False)
          .head(N)[zs]
          .stack()
          .sort_values(axis='index', ascending=False)
          .keys()
    )

    # Build the first guess solely by sorting based on relative occurences per register
    guess = {}
    for letter, reg in occurrences_by_register:
        if reg not in guess:
            if letter not in guess.values():
                guess[reg] = letter

    import operator
    f = operator.itemgetter(0)
    out = list(guess.items())
    out.sort(key=lambda x: x[0])
    res = ''.join([l for _,l in out])
    return res

res = first_guess(df)
print(res)

oarie


In [271]:
import makewords

makewords.possible_words('ibcdefghijklmpqrstuvwxyz', exclude='aon', length=5)

[makewords] Using en wordlist sourced from nltk.


['bebed',
 'becry',
 'becut',
 'bedel',
 'bedew',
 'bedim',
 'bedip',
 'bedub',
 'bedur',
 'bedye',
 'beech',
 'beefy',
 'beery',
 'beest',
 'beeth',
 'beety',
 'beeve',
 'befit',
 'begem',
 'beget',
 'begum',
 'begut',
 'beice',
 'beige',
 'bejel',
 'bejig',
 'belch',
 'belee',
 'belie',
 'belle',
 'belly',
 'belve',
 'bemix',
 'bemud',
 'beret',
 'bergy',
 'berri',
 'berry',
 'berth',
 'beryl',
 'besee',
 'beset',
 'besit',
 'bespy',
 'betel',
 'betis',
 'betty',
 'bevel',
 'bever',
 'bevue',
 'bewet',
 'bewig',
 'bezel',
 'bezzi',
 'bichy',
 'biddy',
 'bider',
 'bidet',
 'bidri',
 'bield',
 'bifer',
 'bifid',
 'bight',
 'bilby',
 'bilch',
 'bilge',
 'bilgy',
 'bilic',
 'billy',
 'bilsh',
 'biped',
 'birch',
 'birdy',
 'birle',
 'birse',
 'birsy',
 'birth',
 'bisti',
 'bitch',
 'biter',
 'bitty',
 'bizet',
 'bleck',
 'bleed',
 'bless',
 'blest',
 'blibe',
 'blick',
 'blimp',
 'blimy',
 'bliss',
 'blite',
 'blitz',
 'blizz',
 'bluer',
 'blues',
 'bluet',
 'bluey',
 'bluff',
 'blurb',


In [272]:
makewords.possible_words('ibdefhijkmpqrstvwxyz', exclude='aonclug', length=5)

[makewords] Using en wordlist sourced from nltk.


['bebed',
 'bedew',
 'bedim',
 'bedip',
 'bedye',
 'beefy',
 'beery',
 'beest',
 'beeth',
 'beety',
 'beeve',
 'befit',
 'bemix',
 'beret',
 'berri',
 'berry',
 'berth',
 'besee',
 'beset',
 'besit',
 'bespy',
 'betis',
 'betty',
 'bever',
 'bewet',
 'bezzi',
 'biddy',
 'bider',
 'bidet',
 'bidri',
 'bifer',
 'bifid',
 'biped',
 'birdy',
 'birse',
 'birsy',
 'birth',
 'bisti',
 'biter',
 'bitty',
 'bizet',
 'brede',
 'bredi',
 'breed',
 'breek',
 'breme',
 'breth',
 'brett',
 'breve',
 'bribe',
 'bride',
 'brief',
 'brier',
 'brisk',
 'briss',
 'brith',
 'brizz',
 'debby',
 'debit',
 'deedy',
 'defer',
 'deify',
 'deism',
 'deist',
 'deity',
 'demit',
 'depth',
 'derby',
 'derry',
 'desex',
 'deter',
 'dewer',
 'dheri',
 'diddy',
 'didie',
 'didst',
 'didym',
 'diker',
 'dimer',
 'dimit',
 'dimps',
 'dirty',
 'disme',
 'diter',
 'ditty',
 'diver',
 'divvy',
 'dixie',
 'dixit',
 'dizzy',
 'dreep',
 'dress',
 'drest',
 'dried',
 'drier',
 'drift',
 'drisk',
 'drive',
 'dryth',
 'dyker',
